In [4]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import os
import time

filename='Dataset_de_200_Comunidades_para_Analise_Ambiental.csv'
df= pd.read_csv(filename)


# Inicializar o geocodificador Nominatim
geolocator = Nominatim(user_agent="geo_localizador_comunidades")

def get_coordinates(city_name, state='Pará', country='Brasil', max_retries=3, timeout=5):
    for attempt in range(max_retries):
        try:
            location = geolocator.geocode(f"{city_name}, {state}, {country}", timeout=timeout)
            if location:
                return location.latitude, location.longitude
            return None, None
        except GeocoderTimedOut:
            print(f"Timeout ao geocodificar {city_name}, tentativa {attempt + 1}/{max_retries}...")
            time.sleep(2 ** attempt)  # Espera exponencial
        except GeocoderServiceError as e:
            print(f"Erro de serviço ao geocodificar {city_name}: {e}")
            return None, None
        except Exception as e:
            print(f"Erro inesperado ao geocodificar {city_name}: {e}")
            return None, None
    print(f"Falha ao geocodificar {city_name} após {max_retries} tentativas.")
    return None, None

# Armazenar latitude e longitude
latitudes = []
longitudes = []

# Obter as coordenadas para cada município único no DataFrame
municipios_unicos = df['Município'].unique()
coordenadas_municipios = {}

print("Obtendo coordenadas dos municípios:")
for municipio in municipios_unicos:
    if municipio not in coordenadas_municipios:
        coords = get_coordinates(municipio)
        if coords:
            coordenadas_municipios[municipio] = coords
            print(f"Coordenadas de {municipio}: Latitude={coords[0]:.2f}, Longitude={coords[1]:.2f}")
        else:
            print(f"Não foi possível encontrar as coordenadas de {municipio}")
    else:
        print(f"Coordenadas de {municipio} já foram obtidas.")

# Adicionar latitude e longitude dos municipios 
print("\nAdicionando coordenadas ao DataFrame:")
for index, row in df.iterrows():
    municipio = row['Município']
    if municipio in coordenadas_municipios:
        lat, lon = coordenadas_municipios[municipio]
        latitudes.append(lat)
        longitudes.append(lon)
        #print(f"Comunidade {row['Comunidade']} ({municipio}): Latitude={lat:.2f}, Longitude={lon:.2f}")
    else:
        latitudes.append(None)
        longitudes.append(None)
        #print(f"Comunidade {row['Comunidade']} ({municipio}): Coordenadas não encontradas.")

# Adicionar as novas colunas ao DataFrame
df['Latitude'] = latitudes
df['Longitude'] = longitudes

# Exibir o DataFrame com as colunas de latitude e longitude
#print("\nDataFrame com Latitudes e Longitudes:")

counter = 1
while os.path.exists(filename):
    filename = f'Dataset_de_200_Comunidades_para_Analise_Ambiental{counter}.csv'
    counter += 1

print('Salvando o arquivo')
# Salvar o DataFrame com o nome único
df.to_csv(filename, index=False, encoding='utf-8')


Obtendo coordenadas dos municípios:
Coordenadas de SÃO DOMINGOS DO CAPIM: Latitude=-1.68, Longitude=-47.77
Coordenadas de SALINÓPOLIS: Latitude=-0.63, Longitude=-47.34
Coordenadas de TRACUATEUA: Latitude=-1.08, Longitude=-46.90
Coordenadas de SÃO MIGUEL DO GUAMÁ: Latitude=-1.61, Longitude=-47.48
Coordenadas de MAGALHÃES BARATA: Latitude=-0.80, Longitude=-47.60
Coordenadas de PALESTINA DO PARÁ: Latitude=-5.74, Longitude=-48.32
Coordenadas de CURIONÓPOLIS: Latitude=-6.10, Longitude=-49.61
Coordenadas de MARAPANIM: Latitude=-0.71, Longitude=-47.70
Coordenadas de PONTA DE PEDRAS: Latitude=-1.39, Longitude=-48.87
Coordenadas de XINGUARA: Latitude=-7.10, Longitude=-49.94
Coordenadas de SANTARÉM NOVO: Latitude=-0.93, Longitude=-47.40
Coordenadas de SÃO FÉLIX DO XINGU: Latitude=-6.64, Longitude=-51.98
Coordenadas de SANTA MARIA DAS BARREIRAS: Latitude=-8.87, Longitude=-49.72
Coordenadas de CAPANEMA: Latitude=-1.19, Longitude=-47.18
Coordenadas de BANNACH: Latitude=-7.35, Longitude=-50.41
Coord